In this notebook we will modulate and transmit QAM symbols using OFDM. The transmit signal will be sent through a frequency selective channel, modeled as a *tapped delay line*. Even though this channel is frequency selective, OFDM make the channel appears flat at each subcarrier, although each subcarrier has a different channel gain that we will need to compensate. This compensasion is equivalent to a 1-tap equalizer and its simplicity is one of the main OFDM advantages.

First we do some initializations and import some packages we will use.

In [ ]:
%matplotlib inline

In [ ]:
import sys
sys.path.append('/home/darlan/cvs_files/pyphysim/')
import math
from matplotlib import pyplot as plt
from pyphysim.comm import channels
from pyphysim.comm.ofdm import OFDM
from pyphysim.comm.modulators import QAM
from pyphysim.util.misc import randn_c

Now we can set the simulation parameters.

In [ ]:
noise_var = 1e-3
bandwidth = 5e6    # 20 MHz bandwidth
Fd = 0             # Doppler frequency (in Hz)
Ts = 1./bandwidth  # Sampling interval
fft_size = 1024
num_used_subcarriers = 600
num_symbols = 10*num_used_subcarriers
cp_size = 10

Now we can create the QAM and OFDM objects and generate the transmit signal.

In [ ]:
# Creates the required objects
qam = QAM(16)
ofdm = OFDM(fft_size, cp_size, num_used_subcarriers)

# Generate some random data
data = np.random.randint(0,16, num_symbols)

# Modulate the data
qam_symbols = qam.modulate(data)

# OFDM Modulate the QAM symbols
ofdm_symbols = ofdm.modulate(qam_symbols)

Let's see the QAM symbols in a scatter plot. Since the symbols were not corrupted yet all symbols should only be one of the 16 possible symbols in the QAM-16 constellation. Also, note that the mean power of the symbols is close to 1.

In [ ]:
plt.plot(np.real(qam_symbols), np.imag(qam_symbols), 'r*')
plt.title('QAM symbols')
plt.show()

# The QAM symbols have an average power equal to 1.0
print("Mean symbol power: {0}".format(np.mean(np.abs(qam_symbols)**2)))

In [ ]:
plt.plot(np.real(ofdm_symbols), np.imag(ofdm_symbols), 'r*')
plt.show()

# The power is divided among all subcarriers. 
# The whole OFMD still has an average power equal to 1.0.
print("Mean symbol power: {0}".format(
        np.mean(np.abs(ofdm_symbols)**2)))

In [ ]:
# Create a jakes object with 20 rays. 
# This will be passed to the TDL channel object.
jakesObj = channels.JakesSampleGenerator(Fd, Ts, L=20)
tdlchannel = channels.TdlChannel.create_from_channel_profile(
    jakesObj, channels.COST259_RAx)

In [ ]:
fading_map = tdlchannel.get_fading_map(ofdm_symbols.size)

In [ ]:
print(fading_map.shape)
plt.stem(np.abs(fading_map[:,0]))
plt.show()

In [ ]:
# Transmit the ofdm modulated signal through the TDL channel
received_ofdm_symbols = tdlchannel.transmit_signal_with_known_fading_map(
    ofdm_symbols, fading_map)

received_ofdm_symbols += math.sqrt(noise_var) * randn_c(
    received_ofdm_symbols.size)

In [ ]:
ofdm_demodulated_data = ofdm.demodulate(
    received_ofdm_symbols[0:ofdm_symbols.size])

ofdm_demodulated_data = np.reshape(ofdm_demodulated_data, [-1, num_used_subcarriers], order='C')

received_ofdm_symbol1 = ofdm_demodulated_data[0]
received_ofdm_symbol2 = ofdm_demodulated_data[1]

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
plt.plot(np.real(received_ofdm_symbol1), np.imag(received_ofdm_symbol1), 'r*')
plt.title('First demodulated OFDM symbol')

plt.subplot(2,1,2)
plt.plot(np.real(received_ofdm_symbol2), np.imag(received_ofdm_symbol2), 'r*')
plt.title('Second demodulated OFDM symbol')
plt.show()

In [ ]:
# Get the channel frequency response
full_fading_map = tdlchannel.include_the_zeros_in_fading_map(fading_map)
channel_freq_resp = tdlchannel.get_channel_freq_response(
    full_fading_map, fft_size)

# Mean (delay domain average) frequency response
mean_freq_response = np.mean(channel_freq_resp, axis=1)

num_unused_subcarriers = fft_size - num_used_subcarriers

# Note that we need to use fftshift to reorder the frequency 
# response into the natural subcarrier order
mean_freq_response = np.fft.fftshift(mean_freq_response)

In [ ]:
# Plot channel frequency response at different time samples
plt.plot(np.abs(channel_freq_resp[:,0]))
plt.hold(True)
plt.plot(np.abs(channel_freq_resp[:,0]))
plt.plot(np.abs(channel_freq_resp[:,512]))
plt.plot(np.abs(channel_freq_resp[:,1024]))
plt.legend(['Sample 0', 'Sample 512', 'Sample 1024'])
plt.xlim([0,1024])
plt.show()

In [ ]:
equalized_ofdm_demodulated_data = ofdm_demodulated_data \
    / mean_freq_response[np.newaxis, num_unused_subcarriers//2:-num_unused_subcarriers//2]
equalized_ofdm_demodulated_data = equalized_ofdm_demodulated_data.flatten()

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(np.real(equalized_ofdm_demodulated_data), np.imag(equalized_ofdm_demodulated_data), 'r*')
plt.title('QAM symbols transmitted in the OFDM symbols')
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.show()

In [ ]:
# Compare transmitted data with received data
#received_qam_symbols = np.hstack(
 #   [equalized_received_ofdm_symbol1, equalized_received_ofdm_symbol2])

received_data = qam.demodulate(equalized_ofdm_demodulated_data)

# Compare received data with transmitted data
ser = 1 - np.sum(data == received_data) / data.size
print("Symbol Error Rate: {0}".format(ser))

## Convolution as Matrix Multiplication

Remove this section if you don't finish it.

In [ ]:
from scipy.linalg import circulant, toeplitz

In [ ]:
num_channel_taps = fading_map.shape[0]
num_zero_padding = fft_size - num_channel_taps
print("Number of channel taps: {0}".format(num_channel_taps))

print("Number of zero padding: {0}".format(num_zero_padding))

$$(f * g)[n]\ \stackrel{\mathrm{def}}{=}\ \sum_{m=-\infty}^\infty f[m]\, g[n - m]$$

$$(f*g)[n]=\sum_{m=-M}^M f[n-m]g[m]$$



In [ ]:

h = fading_map[:,0]

H = toeplitz(h, np.zeros(h.size))
x = np.array([1,2,3, 0])


#[1, 2, 3, 0, 0, 0] * h[0]
#[0, 1, 2, 3, 0, 0] * h[1]
#[0, 0, 1, 2, 3, 0] * h[2]
#[0, 0, 0, 1, 2, 3] * h[3]
    
#out = np.zeros(3+4-1, dtype=complex)
#out[0] = x[0]*0 + 

print(H.dot(x))
print(np.convolve(x,h, mode='full'))

